In [1]:
import json
with open("sample_data.json","r") as inputfile:
    sample_data = json.load(inputfile)

In [2]:
for dataset in sample_data.keys():
    print(f"***{dataset}***")
    for k, v in sample_data[dataset].items():
        print(k,"||", v)

***sick***
A tan dog is playing in the water on the bank of a pond || ∀x∀y∀z (TanDog(x) ∧ Water(y) ∧ Pond(z) → PlaysIn(x, y, z))
Some water is being drunk by a cat || ∃x∃y (Water(x) ∧ Cat(y) ∧ Drinking(y, x))
A person is playing a keyboard || ∀x (Person(x) ∧ PlayingKeyboard(x))
The man is sitting outdoors under a purple umbrella || ∃x (Man(x) ∧ Sitting(x) ∧ Outdoors(x) ∧ Under(x, purple_umbrella))
A man with no shirt is holding a football || ∀x (Man(x) ∧ ¬HasShirt(x) → HoldingFootball(x))
There is no person in bike gear standing steadily in front of the mountains || ¬∃x (Person(x) ∧ InBikeGear(x) ∧ StandingSteadilyInFrontOf(x, mountains))
A woman is slicing a tomato || ∃x (Woman(x) ∧ Slicing(x, tomato))
A few people are singing || ∃x∃y∃z (Person(x) ∧ Person(y) ∧ Person(z) ∧ Singing(x) ∧ Singing(y) ∧ Singing(z))
The surfer is riding a big wave || ∃x ∃y (Surfer(x) ∧ Wave(y) ∧ Riding(x, y))
Two dogs and two people are walking in a wood || ∃x ∃y ∃z ∃w (Dog(x) ∧ Dog(y) ∧ Person(z) ∧ Person(

In [3]:
# Imports 
import re
from nltk.sem.logic import Expression, Variable 

In [4]:
# Convert FOL with classic symbols to NLTK readable FOL 
def fol2nltk(s:str) -> str:
        s = re.sub('∧', '&', s)
        s = re.sub('∃(\S\d*)', 'exists \\1.', s) #counts all chars after quantifier until there is white space as the var  
        s = re.sub('∀(\S\d*)', 'all \\1.', s)
        s = re.sub('→', '->', s)
        s = re.sub('¬', 'not ', s)
        return s 

In [5]:
# Get a test sentence from the sample data set 
test_sent = sample_data['sick']['A person is playing a keyboard']
test_sent = fol2nltk(test_sent)
print(test_sent)

all x. (Person(x) & PlayingKeyboard(x))


In [6]:
# read string of expression into NLTK 
#! Expression.fromstring errors when:
    # unmatched brackets, 
    # var is not of form "letter(+any number)"
 
# test_expr = Expression.fromstring(test_sent) #option with expression from sample data set 
test_expr = Expression.fromstring('exists x. (Red(book) & (all y. Ball(y)) | House(z))') #option with custom expression 


In [7]:
# info about expression object 
print(type(test_expr))
print(dir(test_expr))

<class 'nltk.sem.logic.ExistsExpression'>
['__and__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__neg__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_logic_parser', '_set_type', '_type_checking_logic_parser', 'alpha_convert', 'applyto', 'constants', 'equiv', 'findtype', 'free', 'fromstring', 'getQuantifier', 'make_VariableExpression', 'negate', 'normalize', 'predicates', 'replace', 'simplify', 'substitute_bindings', 'term', 'type', 'typecheck', 'variable', 'variables', 'visit', 'visit_structured']


In [ ]:
#Get some documentation of available functions of an expressions object 
for func in dir(Expression):
    if not func.startswith('_'):
        print(f"************ {func} ************")
        help('nltk.sem.logic.Expression.' + func)
        

In [9]:
# Get info on an expression 

#get all free variables as set of nltk.sem.logic.Variable objects 
free_vars = test_expr.free()
print(f'free variables in "{test_expr}": {free_vars}')
print()

# Get main(?) quantifier
quantifiers = test_expr.getQuantifier()
print(f'quantifiers in "{test_expr}": {quantifiers}')
print()

# Get all constants as set of nltk.sem.logic.Variable objects 
constants = test_expr.constants()
print(f'Constants in "{test_expr}": {constants}')
print()

# Get all predicates as set of nltk.sem.logic.Variable objects 
predicates = test_expr.predicates()
print(f'Predicates in "{test_expr}": {predicates}')

free variables in "exists x.((Red(book) & all y.Ball(y)) | House(z))": {Variable('z')}

quantifiers in "exists x.((Red(book) & all y.Ball(y)) | House(z))": exists

Constants in "exists x.((Red(book) & all y.Ball(y)) | House(z))": {Variable('book')}

Predicates in "exists x.((Red(book) & all y.Ball(y)) | House(z))": {Variable('House'), Variable('Ball'), Variable('Red')}


In [10]:
# Replace predicates (and more?) in an expression 
old_pred = Variable('Red')
new_pred = Expression.fromstring('Blue')
new_test_expr = test_expr.replace(old_pred, new_pred)
print(f'Changed "{test_expr}" to "{new_test_expr}"')

Changed "exists x.((Red(book) & all y.Ball(y)) | House(z))" to "exists x.((Blue(book) & all y.Ball(y)) | House(z))"


In [11]:
#Replace names of FREE variables 
bindings = {
    Variable('x'): Expression.fromstring('a'),
    Variable('y'): Expression.fromstring('b'),
    Variable('z'): Expression.fromstring('c')
}
print(test_expr.variables())
new_test_expr2 = test_expr.substitute_bindings(bindings)
print(f'Changed "{test_expr}" to "{new_test_expr2}"')

{Variable('z')}
Changed "exists x.((Red(book) & all y.Ball(y)) | House(z))" to "exists x.((Red(book) & all y.Ball(y)) | House(c))"


In [12]:
# Get et types of the sentence 

#all et types returnd as: dict{'word': et-type}
et_types = test_expr.typecheck()
print(f"The et types of the expression {test_expr} are: {et_types}")

# get et-type of specific variable 
var = Variable('Red')
et_type_var = test_expr.findtype(var)
print(f"The et type of {var} in the expression {test_expr} is: {et_type_var}")

The et types of the expression exists x.((Red(book) & all y.Ball(y)) | House(z)) are: {'book': e, 'Red': <e,t>, 'y': e, 'Ball': <e,t>, 'z': e, 'House': <e,t>}
The et type of Red in the expression exists x.((Red(book) & all y.Ball(y)) | House(z)) is: <e,t>


In [13]:
#Get all predicates in a specific data set

all_predicates = []
for item in sample_data['sick'].values():
    expression = Expression.fromstring(fol2nltk(item))
    predicates = [str(p) for p in list(expression.predicates())]
    # print(predicates)
    all_predicates = all_predicates + predicates
    # print(all_predicates)
all_predicates = set(all_predicates) 
print(all_predicates)


{'Water', 'Pond', 'Cat', 'StandingSteadilyInFrontOf', 'Play', 'Slicing', 'Wave', 'Woman', 'HoldingFootball', 'CuttingOnion', 'TanDog', 'WalkingInWood', 'Man', 'Outdoors', 'Sitting', 'Riding', 'HasShirt', 'Kickboxing', 'Surfer', 'Dog', 'Under', 'Person', 'Singing', 'InBikeGear', 'Microphone', 'Spectator', 'HappyBoy', 'SingingInto', 'Drinking', 'PlaysIn', 'PlayingKeyboard', 'PlaysPiano'}


In [26]:
new_predicates = []
for p in all_predicates:
    parts = re.findall('[A-Z][^A-Z]*', p)
    split_p = [parts[0]]
    if len(parts) > 1:
        second_pred =""
        for part in parts[1:]:
            second_pred = second_pred + part 
        split_p.append(second_pred)
    new_predicates += split_p
print(new_predicates)
# test_expr = Expression.fromstring('exist x. HoldingFootball(x)')
# old_pred = Variable('HoldingFootball')
# new_pred = Expression.fromstring('Holding(x, y) & Football(y)')
# new_test_expr = test_expr.replace(old_pred, new_pred)
# print(f'Changed "{test_expr}" to "{new_test_expr}"')

['Water', 'Pond', 'Cat', 'Standing', 'SteadilyInFrontOf', 'Play', 'Slicing', 'Wave', 'Woman', 'Holding', 'Football', 'Cutting', 'Onion', 'Tan', 'Dog', 'Walking', 'InWood', 'Man', 'Outdoors', 'Sitting', 'Riding', 'Has', 'Shirt', 'Kickboxing', 'Surfer', 'Dog', 'Under', 'Person', 'Singing', 'In', 'BikeGear', 'Microphone', 'Spectator', 'Happy', 'Boy', 'Singing', 'Into', 'Drinking', 'Plays', 'In', 'Playing', 'Keyboard', 'Plays', 'Piano']


In [24]:
# split predicates 
result = re.findall('[A-Z][^A-Z]*', 'HoldingFootballBlue')
result = [result[0], result[1:]]
print(result)

['Holding', ['Football', 'Blue']]
